In [3]:
import os
import shutil
import weaviate
from weaviate.classes.config import Configure
from langchain_community.document_loaders import PyPDFLoader, TextLoader, JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_weaviate.vectorstores import WeaviateVectorStore
from langchain_ollama import OllamaEmbeddings

In [4]:
DATA_DIR = '../data'

UPLOAD_DIR = "uploads"

UPLOAD_PATH = os.path.join(DATA_DIR, UPLOAD_DIR)

os.makedirs(UPLOAD_PATH, exist_ok=True)

In [5]:
weaviate_client = weaviate.connect_to_local()

embeddings = OllamaEmbeddings(
    model="nomic-embed-text",
)

In [6]:
input_text = "The meaning of life is 42"
vector = embeddings.embed_query(input_text)
print(vector[:3])

[0.008603463, -0.008584372, -0.15307632]


In [7]:
vec_db = WeaviateVectorStore(
    client=weaviate_client,
    index_name="LangChain_0b0b3243518a4c268131915f0ebe04cb",
    text_key='text',
    embedding=embeddings
)

In [8]:
from pydantic import BaseModel
from typing import List, Dict, Optional

from langchain import hub
from langchain_ollama import ChatOllama


prompt = hub.pull("rlm/rag-prompt")


class State(BaseModel):
    question: str
    context: Optional[List[Dict]]
    answer: str
    tenant: str


llm = ChatOllama(
    model="llama3.2:3b",
    temperature=0,
)


def retrieve(db, state: State):
    retrieved_docs = db.similarity_search(state["question"], tenant=state['tenant'])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke(
        {"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

/Users/anurags/Library/Caches/pypoetry/virtualenvs/book-rag-o4xYDdTl-py3.12/lib/python3.12/site-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [9]:
state = {'question': "hottest planet", 'context': None, 'answer': '', 'tenant': "solar-system"}

context = retrieve(vec_db, state)
state['context'] = context['context']

# Print the first 100 characters of each result
for i, doc in enumerate(context['context']):
    print(f"\nDocument {i+1}:")
    # print(doc.page_content[:100] + "...")
    print(doc.page_content)


Document 1:
Venus is the hottest planet.

Document 2:
There are 4 planets made of gas.

Document 3:
It is made of rock and the others are gas planets.

Document 4:
Jupiter has a small core and has a thick layer of gas around it.


In [10]:
print(generate(state)['answer'])

Venus is the hottest planet, with surface temperatures reaching as high as 462°C (863°F). It is composed primarily of rock and metal, unlike the other gas giants in our solar system. Jupiter is not considered the hottest planet.


In [15]:
%%timeit

state = {'question': "hottest planet", 'context': None, 'answer': '', 'tenant': "solar-system"}

context = retrieve(vec_db, state)
state['context'] = context['context']

199 ms ± 39.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Benchmarks Vector Stores
### FAISS
- 179 ms ± 26.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
- 185 ms ± 26.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

### Weaviate (Docker)
- 500 ms ± 263 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
- 1.2 s ± 693 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
- 1.13 s ± 265 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
- 170 ms ± 25.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
- 199 ms ± 39.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
